In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE68606"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE68606"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE68606.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE68606.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE68606.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from Affymetrix Human Genome U133A arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Kidney_Chromophobe)
# Looking at the histology field (index 7), we can see there is data for "Recurrent Renal Cell Carcinoma, chromophobe cell type"
trait_row = 7

# Age data is available in the sample characteristics
age_row = 6

# Gender data is available as "Sex"
gender_row = 5

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert histology information to binary trait (Kidney_Chromophobe)"""
    if value is None or value == "--":
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value mentions chromophobe kidney cancer
    if "chromophobe" in value.lower() and "renal" in value.lower():
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age information to continuous values"""
    if value is None or value == "--":
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender information to binary (0=female, 1=male)"""
    if value is None or value == "--":
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = value.lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Note: We're skipping the clinical feature extraction step since we don't have
# the properly formatted sample-level clinical data required for processing.
# The sample characteristics dictionary only shows unique values for each characteristic,
# not the actual data for each sample.


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 22283 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers '1007_s_at', '1053_at', etc. are Affymetrix probe IDs,
# not standard human gene symbols. These are microarray probe identifiers from
# the Affymetrix GeneChip Human Genome platform.
# They need to be mapped to official gene symbols for standardized analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3075191 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify mapping columns based on previous output
# In gene expression data, the identifiers are Affymetrix probe IDs like '1007_s_at'
# From the gene annotation preview, we need 'ID' (for probe IDs) and 'Gene Symbol' columns

print("Creating gene mapping between probe IDs and gene symbols...")

# 2. Extract the gene mapping from gene annotation data
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Created gene mapping with {len(mapping_df)} entries")

# Preview the mapping
print("\nFirst few rows of the probe-to-gene mapping:")
print(preview_df(mapping_df))

# 3. Convert probe-level measurements to gene-level expression
print("\nConverting probe-level data to gene-level expression data...")
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Gene-level expression data created with {len(gene_data.index)} genes")

# Preview the gene expression data
print("\nFirst few rows of gene-level expression data:")
print(preview_df(gene_data))

# Save the gene data to file
print(f"\nSaving gene expression data to {out_gene_data_file}")
# Make directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print("Gene expression data saved successfully")


Creating gene mapping between probe IDs and gene symbols...


Created gene mapping with 21225 entries

First few rows of the probe-to-gene mapping:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Converting probe-level data to gene-level expression data...


Gene-level expression data created with 13830 genes

First few rows of gene-level expression data:
{'GSM1676864': [411.658, 9.93369, 67.9692, 103.083, 253.048], 'GSM1676865': [328.766, 29.7297, 42.2613, 19.8402, 307.313], 'GSM1676866': [283.038, 31.5178, 103.195, 120.743, 301.599], 'GSM1676867': [441.273, 117.207, 34.8804, 233.94, 54.3385], 'GSM1676868': [456.17, 66.0545, 37.4472, 132.972, 292.66], 'GSM1676869': [356.891, 76.3685, 20.8323, 54.8785, 526.452], 'GSM1676870': [370.538, 8.34365, 63.0356, 86.0991, 285.639], 'GSM1676871': [391.2, 77.8802, 70.4974, 86.7128, 242.461], 'GSM1676872': [517.632, 96.8163, 25.5571, 132.283, 157.348], 'GSM1676873': [330.712, 76.7729, 77.6666, 123.381, 88.7403], 'GSM1676874': [567.981, 6309.08, 44.9647, 111.27, 191.89], 'GSM1676875': [451.008, 3399.73, 47.873, 201.969, 184.049], 'GSM1676876': [398.683, 6781.84, 43.8145, 156.659, 111.423], 'GSM1676877': [264.817, 3324.59, 83.583, 67.3656, 95.2435], 'GSM1676878': [316.144, 3792.41, 128.442, 198.628, 286.

Gene expression data saved successfully


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (this was already done in Step 6, but we'll save the normalized version now)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 1. Extract clinical features first (since we didn't do this in earlier steps)
print("\nExtracting clinical features...")
try:
    # From sample characteristics dictionary in Step 1, histology is at index 7
    # We're looking for "chromophobe" in the kidney cancer descriptions
    trait_row = 7  # histology field
    
    def convert_trait(value):
        """Convert histology information to binary trait (Kidney_Chromophobe)"""
        if value is None or value == "--":
            return None
        
        # Extract the value after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        # Check if the value mentions chromophobe kidney cancer
        if "chromophobe" in value.lower() and "renal" in value.lower():
            return 1
        else:
            return 0
    
    # Define gender conversion (from "Sex:" field at index 5)
    gender_row = 5
    
    def convert_gender(value):
        """Convert gender information to binary (0=female, 1=male)"""
        if value is None or value == "--":
            return None
        
        # Extract the value after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        value = value.lower()
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
        else:
            return None
    
    # Define age conversion (from "age:" field at index 6)
    age_row = 6
    
    def convert_age(value):
        """Convert age information to continuous values"""
        if value is None or value == "--":
            return None
        
        # Extract the value after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        try:
            return float(value)
        except:
            return None
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print(f"Clinical data extracted with shape: {clinical_df.shape}")
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical features: {e}")
    is_trait_available = False
    clinical_df = None

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if clinical_df is not None and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("First few clinical sample columns:", list(clinical_df.columns)[:5])
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        trait_count = linked_data[trait].count()
        print(f"Number of samples with trait values: {trait_count}")
        
        if trait_count > 0:
            # 3. Handle missing values systematically
            print("\nHandling missing values...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"After handling missing values, data shape: {linked_data.shape}")
            
            # Check if we still have samples after missing value handling
            if linked_data.shape[0] > 0:
                # 4. Determine whether the trait and demographic features are biased
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            else:
                print("Error: All samples were removed during missing value handling.")
                is_biased = True
        else:
            print("No samples have valid trait values. Dataset cannot be used.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note="Dataset contains kidney tissue samples with histology data including chromophobe renal cell carcinoma"
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 13542 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE68606.csv

Extracting clinical features...
Clinical data extracted with shape: (3, 137)
Clinical data preview:
{'Kidney_Chromophobe': [0.0, 0.0, 0.0, 0.0, 0.0], 'Age': [nan, nan, nan, nan, nan], 'Gender': [nan, nan, 0.0, nan, nan]}
Clinical data saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE68606.csv

Linking clinical and genetic data...
First few clinical sample columns: ['GSM1676864', 'GSM1676865', 'GSM1676866', 'GSM1676867', 'GSM1676868']
First few genetic sample columns: ['GSM1676864', 'GSM1676865', 'GSM1676866', 'GSM1676867', 'GSM1676868']
Linked data shape: (137, 13545)
Number of samples with trait values: 137

Handling missing values...


After handling missing values, data shape: (137, 13545)

Checking for bias in features...
For the feature 'Kidney_Chromophobe', the least common label is '1.0' with 4 occurrences. This represents 2.92% of the dataset.
The distribution of the feature 'Kidney_Chromophobe' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 61.8
  50% (Median): 61.8
  75%: 61.8
Min: 48.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 9 occurrences. This represents 6.57% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...
Dataset not usable for Kidney_Chromophobe association studies. Data not saved.
